## Datasets
* AS (Traditional Chinese) http://www.khngai.com/chinese/charmap/tblbig.php?page=0
* CITYU (Traditional Chinese)
* MSR (Simplified Chinese)
* PKU (Simplified Chinese)


- Each dataset has one train and one dev set
- You can find the train datasets in the training folder
- Your dev set is in the gold folder
- For each dataset use the file with the .utf8 extension
 

In [1]:
import os

path_training='icwb2-data/training'
path_dev='icwb2-data/gold'
path_testing='icwb2-data/testing'

datasets = [path_training, path_dev, path_testing]
def walk_dir(path):
    count=0
    len_ = len(os.listdir(path))
    print("start for path: {}\n".format(path))
    for path, subdir, files in os.walk(path):
        for name in files: 
            x = os.path.join(path,name)
            TraditionalToSimplifiedChinese(x)
            count+=1
            print(">>>{}/{} complete============\n".format(count, len_))
    print("end\n")
    return None

def TraditionalToSimplifiedChinese(input_file):
    '''transforms files'''
    
    name_, format_ = input_file.split(".")
    file_name_input = name_.split('/')[-1]
    
    name_ += '_simplified.' + format_
    file_name_output = name_.split('/')[-1]
    
    bash_ = 'hanzi-convert -s ' + input_file + ' > ' + name_
    os.system(bash_)
    print("Created {}".format(file_name_output))
    #os.system('rm '+ input_file)
    #print("Deleted {}".format(file_name_input))
    return None

#if __main__ == '__name__':
#[walk_dir(i) for i in datasets]
#walk_dir(path_training)

In [162]:
class Preprocess(object):
    def __init__(self):
        pass
        
    
    @staticmethod 
    def deleteTxt(path, extension = '.txt'):
        '''
        Deletes all the .txt files in a given directory
        Only done to get rid of duplicate .txt files as I am using .utf8 files for training
        '''
        print("Deleteing {} files from {}".format(extension, path))
        for path, subdir, files in os.walk(path):
            for name in files: 
                if name.endswith(extension):
                    os.system('rm '+ os.path.join(path,name))
                    print('Succesfully removed: {}'.format(name))
            print("Done\n")
            
        
    def traditionalToSimplified(self):
        '''transoforms a file form Traditional to Simplified Chinese'''
        
        name_, format_ = input_file.split(".")
        file_name_input = name_.split('/')[-1]
        name_ += '_simplified.' + format_
        file_name_output = name_.split('/')[-1]

        bash_ = 'hanzi-convert -s ' + input_file + ' > ' + name_
        os.system(bash_)
        print("Created {}".format(file_name_output))
        
    def walk_dir(path):
        count=0
        len_ = len(os.listdir(path))
        print("start for path: {}\n".format(path))
        for path, subdir, files in os.walk(path):
            for name in files: 
                x = os.path.join(path,name)
                TraditionalToSimplifiedChinese(x)
                count+=1
                print(">>>{}/{} complete============\n".format(count, len_))
        print("end\n")
        return None
    
    
    def encoder(self):
        pass
        
    def FileToBIES(self):
        pass
    
    
    
    def encoder(line, type_of_file, verbose = False):
        '''encodes a single line'''

        new_line = ''

        #uses the +u3000 seperator if the file is 'AS'
        words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
        #print("N of segments:",len(words))

        def bies_format(word):
            '''changes word to BIES format'''
            return 'B'+'I'*int(len(word)-2)+'E' if len(word)>1 else 'S'

        for word in words:
            if verbose: print("this is a word: {} of length {}".format(word, len(word)))
            new_line+=bies_format(word)

        return new_line

    def FileToBIES(file_path):
        '''gives a file in utf converts to a txt in with all lines in the BIES format'''

        name_ = file_path.split(".")[-2] + '_labels'
        type_of_file = name_.split("/")[-1].split('_')[0] #to see if its an 'AS' file

        with open(file_path, 'r', encoding ='utf8') as f1, open(name_+".txt", "w") as f2:
            c=0
            for line in f1:
                c+=1
                print("input line:\t",line.rstrip())
                out = encoder(line, type_of_file)
                print("output line:\t{}".format(out))
                f2.writelines(out+"\n")
                if c==2:
                    break
        return None

In [3]:
Preprocess.deleteTxt('icwb2-data/gold')

Deleteing .txt files from icwb2-data/gold


In [4]:
class BIES(object):
    
    def __init__(self, word):
        self.word = word
        
    
    def encode(self, line, type_of_file, verbose = False):
        '''encodes a single line'''
        new_line = ''
        #uses the +u3000 seperator if the file is 'AS'
        words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
        
        def bies_format(word):
            '''changes word to BIES format'''
            return 'B'+'I'*int(len(word)-2)+'E' if len(word)>1 else 'S'
        
        for word in words:
            new_line+=bies_format(word)

        return new_line
    
    def decode(self):
        pass
    
A = BIES('w')

In [5]:
BIES

__main__.BIES

In [125]:
def encoder(line, type_of_file, verbose = False):
    '''encodes a single line'''
    
    new_line = ''
        
    #uses the +u3000 seperator if the file is 'AS'
    words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
    print("N of segments:",len(words))
    
    def bies_format(word):
        '''changes word to BIES format'''
        return 'B'+'I'*int(len(word)-2)+'E' if len(word)>1 else 'S'
    
    for word in words:
        if verbose: print("this is a word: {} of length {}".format(word, len(word)))
        new_line+=bies_format(word)
        
    return new_line

def FileToBIES(file_path):
    '''gives a file in utf converts to a txt in with all lines in the BIES format'''
    print(file_path, file_path.split("."))
    name_ = file_path.split("/")[-1].split(".")[0] + '_labels'
    type_of_file = name_.split("/")[-1].split('_')[0] #to see if its an 'AS' file
    print("NAME",name_)
    with open(file_path, 'r', encoding ='utf8') as f1, open(name_+".txt", "w") as f2:
        c=0
        for line in f1:
            c+=1
            print("input line:\t",line.rstrip())
            out = encoder(line, type_of_file)
            print("output line:\t{}".format(out))
            f2.writelines(out+"\n")
            if c==2:
                break
    return None

In [126]:
FileToBIES(file_path = '../icwb2-data/training/pku_training.utf8')

../icwb2-data/training/pku_training.utf8 ['', '', '/icwb2-data/training/pku_training', 'utf8']
NAME pku_training_labels
input line:	 迈向  充满  希望  的  新  世纪  ——  一九九八年  新年  讲话  （  附  图片  １  张  ）
N of segments: 31
output line:	BESBESBESSSSSBESBESBIIIESBESBESSSSSBESSSSSS
input line:	 中共中央  总书记  、  国家  主席  江  泽民
N of segments: 13
output line:	BIIESBIESSSBESBESSSBE


In [127]:
'../icwb2-data/training/pku_training.utf8'.split("/")[-1].split(".")[0]

'pku_training'

In [129]:
FileToBIES(file_path = '../icwb2-data/training/msr_training.utf8')

../icwb2-data/training/msr_training.utf8 ['', '', '/icwb2-data/training/msr_training', 'utf8']
NAME msr_training_labels
input line:	 “  人们  常  说  生活  是  一  部  教科书  ，  而  血  与  火  的  战争  更  是  不可多得  的  教科书  ，  她  确实  是  名副其实  的  ‘  我  的  大学  ’  。
N of segments: 65
output line:	SSBESSSSSBESSSSSSSBIESSSSSSSSSSSSSBESSSSSBIIESSSBIESSSSSBESSSBIIESSSSSSSSSBESSSS
input line:	 “  心  静  渐  知  春  似  海  ，  花  深  每  觉  影  生  香  。
N of segments: 33
output line:	SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS


In [9]:
FileToBIES(file_path = 'icwb2-data/training/cityu_training.utf8')

icwb2-data/training/cityu_training.utf8


FileNotFoundError: [Errno 2] No such file or directory: 'icwb2-data/training/cityu_training.utf8'

In [10]:
FileToBIES(file_path = 'icwb2-data/training/as_training.utf8')

icwb2-data/training/as_training.utf8


FileNotFoundError: [Errno 2] No such file or directory: 'icwb2-data/training/as_training.utf8'

In [11]:
FileToBIES(file_path = 'sample_files/sample_icwb2.utf8')

sample_files/sample_icwb2.utf8


FileNotFoundError: [Errno 2] No such file or directory: 'sample_files/sample_icwb2.utf8'

## TO DO
1. read original file in a train/dev/test fodler
2. produce an input file with NO spaces
3. and a label file in BIES format

In [12]:
#delete txt
import os
path_training='icwb2-data/training'
path_dev='icwb2-data/gold'
path_testing='icwb2-data/testing'


path = path_training

for path, subdir, files in os.walk(path):
    for name in files: 
        if name.endswith('.txt'):
            os.system('rm '+ os.path.join(path,name))
            print('Succesfully removed: {}'.format(name))
print("end\n")

end



In [13]:
name.split(".")

NameError: name 'name' is not defined

In [155]:
datasets = {"training":'../icwb2-data/training',
             "dev":'../icwb2-data/gold',
             "testing":'../icwb2-data/testing'}

In [163]:
class Preprocess(object):
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        
    def deleteFiles(self, extension = '.txt'):
        '''
        Deletes all the .txt files in a given directory
        Only done to get rid of duplicate .txt files as I am using .utf8 files for training
        '''
        print("Deleteing {} files from {}".format(extension, self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                if name.endswith(extension):
                    os.system('rm '+ os.path.join(self.dataset_path,name))
                    print('Succesfully removed: {}'.format(name))
            print("Done\n")

    def hanzi_convert(self, input_file):
        '''transoforms a file form Traditional to Simplified Chinese'''
        
        _, _, name_, format_ = input_file.split(".")
        file_name_input = name_.split('/')[-1]
        name_ += '_simplified.' + format_
        file_name_output = name_.split('/')[-1]

        bash_ = 'hanzi-convert -s ' + input_file + ' > ' + name_
        os.system(bash_)
        print("Created {}".format(file_name_output))
        
    def traditionalToSimplified(self):
        count=0
        len_ = len(os.listdir(self.dataset_path))
        print("start for path: {}\n".format(self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                x = os.path.join(self.dataset_path, name)
                self.hanzi_convert(x)
                count+=1
                print(">>>{}/{} complete============\n".format(count, len_))
        print("end\n")
#         return None
    
    def action(self, action_type):
        count=0
        len_ = len(os.listdir(self.dataset_path))
        print("start for path: {}\n".format(self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                x = os.path.join(self.dataset_path, name)
                print("File: {}".format(x))
                if action_type == 'translate': 
                    self.hanzi_convert(x)
                elif action_type == 'InputFile':
                    self.InputFile(x)
                elif action_type == 'LabelFile':
                    self.FileToBIES(x)   
                count+=1
                print(">>>{}/{} complete============\n".format(count, len_))
        print("end\n")
        return None
    
    
    
    def InputFile(self, input_file):
        '''creates a file to feed to the TF model by getting rid of spaces'''
        pass
    
    
    
    
    @staticmethod
    def encoder(line, type_of_file, verbose = False):
        '''encodes a single line'''

        new_line = ''

        #uses the +u3000 seperator if the file is 'AS'
        words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
        #print("N of segments:",len(words))

        def bies_format(word):
            '''changes word to BIES format'''
            return 'B'+'I'*int(len(word)-2)+'E' if len(word)>1 else 'S'

        for word in words:
            if verbose: print("this is a word: {} of length {}".format(word, len(word)))
            new_line+=bies_format(word)

        return new_line

    def FileToBIES(self, file_path):
        '''gives a file in utf converts to a txt in with all lines in the BIES format'''


        #print("HERE:",file_path)
        name_ = file_path.split("/")[-1].split(".")[0] + '_labels'
        #print("NAME",name_, file_path)
        type_of_file = name_.split("/")[-1].split('_')[0] #to see if its an 'AS' file

        with open(file_path, 'r', encoding ='utf8') as f1, open(name_+".txt", "w") as f2:
            c=0
            for line in f1:
                c+=1
                #print("input line:\t",line.rstrip())
                out = encoder(line, type_of_file)
                #print("output line:\t{}".format(out))
                f2.writelines(out+"\n")
        return None
    

In [164]:

P = Preprocess(datasets['training'])
P.deleteFiles('.txt')
P.deletefiles('.b5')
#P.traditionalToSimplified()
P.action('LabelFile')

Deleteing .txt files from ../icwb2-data/training
Done



AttributeError: 'Preprocess' object has no attribute 'deletefiles'

In [165]:
P.deleteFiles('.b5')

Deleteing .b5 files from ../icwb2-data/training
Done



In [166]:
P.traditionalToSimplified()

start for path: ../icwb2-data/training

Created msr_training_simplified.utf8
>>>1/4 complete============

Created cityu_training_simplified.utf8
>>>2/4 complete============

Created as_training_simplified.utf8
>>>3/4 complete============

Created pku_training_simplified.utf8
>>>4/4 complete============

end



In [170]:
help(encoder)

Help on function encoder in module __main__:

encoder(line, type_of_file, verbose=False)
    encodes a single line

